In [1]:
from prettytable import PrettyTable
from matplotlib import pyplot as plot
from math import fabs
import copy

In [2]:
class SourceTable:
    def __init__(self, fn):
        self.array_temperature = []
        self.array_capacity = []
        self.array_k_coeff = []
        for line in fn:
            value_T, value_C, value_k = line.split(',')
            value_T = float(value_T)
            value_C = float(value_C)
            value_k = float(value_k)
            self.array_temperature.insert(len(self.array_temperature), value_T)
            self.array_capacity.insert(len(self.array_capacity), value_C)
            self.array_k_coeff.insert(len(self.array_k_coeff), value_k)

    def get_capacity(self, t):
        index = 0
        if t <= self.array_temperature[0]:
            index = 1
        elif t > self.array_temperature[-1]:
            index = len(self.array_temperature) - 1
        else:
            while self.array_temperature[index] < t:
                index += 1
        return self.array_capacity[index - 1] + (self.array_capacity[index] - 
                                                 self.array_capacity[index - 1]) * (
                       t - self.array_temperature[index - 1]) / (
                self.array_temperature[index] - self.array_temperature[index - 1])

    def get_coeff_k(self, t):
        index = 0
        if t <= self.array_temperature[0]:
            index = 1
        elif t > self.array_temperature[-1]:
            index = len(self.array_temperature) - 1
        else:
            while self.array_temperature[index] < t:
                index += 1
        return self.array_k_coeff[index - 1] + (self.array_k_coeff[index] - 
                                                self.array_k_coeff[index - 1]) * (
                t - self.array_temperature[index - 1]) / (
                self.array_temperature[index] - self.array_temperature[index - 1])

    def __repr__(self):
        return str(self.array_temperature) + '\n' + \
               str(self.array_capacity) + '\n' + \
               str(self.array_k_coeff)


In [3]:
# -- Source data --
rod_radius = 0.5            # Radius of the rod
first_cond_coeff = 0.2      # Parameter for conductivity coefficient
second_cond_coeff = 0.5     # Ditto
begin_transf_coeff = 1e-2   # Heat transfer coefficient (start point)
end_transf_coeff = 9e-3     # Ditto (end point)
rod_length = 10             # Rod length
env_temperature = 300       # Environment temperature
heat_stream_density = 100   # Heat stream density

nodes_number = 100          # Number of nodes
time_step = 5               # Time step
iterations = 25

In [4]:
def get_alpha(x, a, b):
    return a / (x - b)


def get_hi(table, t1, t2):
    return 2 * table.getK(t1) * table.getK(t2) / (table.getK(t1) + table.getK(t2))


In [5]:
# To calculate the coefficients A and C of the Thomas Algorithm:
def get_A_or_C(first_k, second_k):
    return 2 * first_k * second_k / (first_k + second_k)


def f(x, a, b, t, r):
    return 2 * get_alpha(x, a, b) * t / r


def pf(x, a, b, r):
    return 2 * get_alpha(x, a, b) / r


In [6]:
class ThomasAlgorithmCoefficients:
    def __init__(self):
        self.A = []
        self.B = []
        self.C = []
        self.F = []

        
class BoundaryConditionParameters:
    def __init__(self):
        self.coefficients = ThomasAlgorithmCoefficients()
        
        self.K0 = 0   # For the boundary condition on the left
        self.M0 = 0   # For the boundary condition on the left
        self.P0 = 0   # For the boundary condition on the left
        
        self.KN = 0   # For the boundary condition on the right
        self.MN = 0   # For the boundary condition on the right
        self.PN = 0   # For the boundary condition on the right
        
        self.n = 0    # The number of equations

In [7]:
def runthru_coefficients(data, n, t):
    # Runthru coefficients:
    xi = [0] * (n + 1)
    et = [0] * (n + 1)
    xi[1] = -data.M0 / data.K0
    et[1] = data.P0 / data.K0
    # Forward run:
    for i in range(2, n + 1):
        xi[i] = data.coefficients.C[i - 1] / (data.coefficients.B[i - 1] - data.coefficients.A[i - 1] * xi[i - 1])
        et[i] = (data.coefficients.F[i - 1] + data.coefficients.A[i - 1] * et[i - 1]) / (data.coefficients.B[i - 1] - data.coefficients.A[i - 1] * xi[i - 1])
    t[n] = (data.PN - data.MN * et[n]) / (data.KN + data.MN * xi[n])
    # Reverse run:
    for i in range(n - 1, -1, -1):
        t[i] = xi[i + 1] * t[i + 1] + et[i + 1]
    pass

In [8]:
def calculate():
    b_coeff = end_transf_coeff * rod_length / (end_transf_coeff - begin_transf_coeff)
    a_coeff = -1 * begin_transf_coeff * b_coeff
    h = rod_length / (iterations - 1)
    tau = time_step
    temp_env = env_temperature
    temperature = [[temp_env] * (iterations + 1)]  # Our array becomes 3D for time
    array_for_x = []
    array_for_x.append(0)
    x_value = 0
    # Filling arrays for runthru:
    for i in range(1, iterations + 1):
        x_value += h
        array_for_x.append(x_value)

    wolfram_data = SourceTable(open('./wolfram.txt'))

    # External loop: changing time variable
    while True:
        previous_temperature = temperature[-1]
        new_temperature = copy.copy(temperature[-1])
        while True:
            previous_temperature = copy.copy(new_temperature)
            run_data = BoundaryConditionParameters()
            run_data.n = iterations

            # Getting boundary conditions:
            hi_middle = get_hi(wolfram_data, new_temperature[0], new_temperature[1])
            p_begin = pf(0, a_coeff, b_coeff, rod_radius)
            p_middle = (p_begin + pf(h, a_coeff, b_coeff, rod_radius)) / 2
            capacity_begin = wolfram_data.get_capacity(new_temperature[0])
            capacity_middle = (capacity_begin +
                               wolfram_data.get_capacity(new_temperature[1])) / 2
            f0 = f(0, a_coeff, b_coeff, temp_env, rod_radius)
            f1 = f(h, a_coeff, b_coeff, temp_env, rod_radius)
            run_data.K0 = tau * (hi_middle / h + h * p_middle /
                                 8 + h * p_begin / 4) + h * capacity_begin / 4 + h * capacity_middle / 8
            run_data.M0 = tau * (h * p_middle / 8 - hi_middle / h) + h * capacity_middle / 8
            run_data.P0 = heat_stream_density * tau + h * tau / 8 * (3 * f0 + f1) + h * capacity_begin * \previous_temperature[0] / 4 + h * capacity_middle * (previous_temperature[0] + previous_temperature[1]) / 8

            # Right boundary conditions:
            capacity_N = wolfram_data.get_capacity(new_temperature[iterations])
            capacity_middle = (capacity_N + wolfram_data.get_capacity(new_temperature[iterations])) / 2
            hiN_05 = get_hi(wolfram_data, new_temperature[iterations - 1], new_temperature[iterations])
            pN = pf(rod_length, a_coeff, b_coeff, rod_radius)
            pN_05 = (pN + pf(rod_length - h, a_coeff, b_coeff, rod_radius)) / 2
            fN0 = f(rod_length, a_coeff, b_coeff, temp_env, rod_radius)
            fN1 = f(rod_length - h, a_coeff, b_coeff, temp_env, rod_radius)

            run_data.KN = -tau * (hiN_05 / h + get_alpha(rod_length, a_coeff,
                                                     b_coeff) + pN * h / 4 + pN_05 * h / 8) + h * capacity_N / 4 + h * capacity_middle / 8;
            run_data.MN = tau * (hiN_05 / h - h * pN_05 / 8) + h * capacity_middle / 8
            run_data.PN = -get_alpha(rod_length, a_coeff, b_coeff) * temp_env * tau - tau * h / 8 * (
                    3 * fN0 + fN1) + h / 4 * capacity_N * previous_temperature[iterations] + h / 8 * capacity_middle * (
                                  previous_temperature[iterations] + previous_temperature[iterations- 1])

            # Forming three-diagonal matrix method coefficients:
            array_for_x = []
            run_data.coefficients.A.append(0)
            run_data.coefficients.B.append(0)
            run_data.coefficients.C.append(0)
            run_data.coefficients.F.append(0)
            
            x_value = h
            for i in range(1, iterations):
                array_for_x.append(x_value)
                run_data.coefficients.A.append(tau * get_hi(wolfram_data, new_temperature[i - 1], new_temperature[i]) / h)
                run_data.coefficients.C.append(tau * get_hi(wolfram_data, new_temperature[i], new_temperature[i + 1]) / h)
                run_data.coefficients.B.append(run_data.coefficients.A[-1] + run_data.coefficients.C[-1] + pf(x_value, a_coeff, b_coeff,
                                                                                         rod_radius) * h * tau + wolfram_data.get_capacity(
                    new_temperature[i]) * h)
                run_data.coefficients.F.append(
                    wolfram_data.get_capacity(new_temperature[i]) * h * previous_temperature[i] + f(x_value, a_coeff, b_coeff,
                                                                                            temp_env,
                                                                                            rod_radius) * h * tau)
                x_value += h
            array_for_x.append(x_value)
            runthru_coefficients(run_data, iterations, new_temperature)
            difference_max = 0
            in_iteration_max = 0
            for i in range(0, len(new_temperature)):
                if fabs(new_temperature[i] - previous_temperature[i]) > difference_max:
                    difference_max = fabs(new_temperature[i] - previous_temperature[i])
                    in_iteration_max = i
            if fabs(difference_max / new_temperature[in_iteration_max]) < 1e-4:
                break
            difference_max = 0
            in_iteration_max = 0
            for i in range(0, len(new_temperature)):
                if fabs(new_temperature[i] - previous_temperature[i]) > difference_max:
                    difference_max = fabs(new_temperature[i] - previous_temperature[i])
                    in_iteration_max = i
            temperature.append(new_temperature)
            dif = fabs(difference_max) / temperature[len(temperature) - 2][in_iteration_max]
    print(dif, temperature[-1][in_iteration_max])
    if dif < 3 * 1e-6:
                print('Done!')
                break
    return [temperature, array_for_x]


In [ ]:
def calculateThing_Stat():
    b = testdata.aN*testdata.l / (testdata.aN-testdata.a0)
    a = -1 * testdata.a0 * b
    h = testdata.l / (testdata.N - 1)
    tau = testdata.tau
    Tenv = testdata.Tenv
    n = testdata.N
    T = [[Tenv]*(testdata.N+1)] # our array becomes 3D for time
    x = []
    x.append(0);
    xval = 0;
    # Filling arrays for runthru
    for i in range(1, n+1):
        xval+=h
        x.append(xval);

    mattable = Datatable.sTable(open('./wolfram_stat.txt'))
    dmax = 0;
    imax = 0;
    Told = Tenv;
    TPast = T[-1]
    T_new = copy.copy(T[-1])
    while True:
        T_old = copy.copy(T_new)
        rundata = run_diagonal.data()
        rundata.n = n
        # Getting boundary conditions:
        hi_05 = hi(mattable, T_new[0], T_new[1])
        p0 = pf(0,a,b,testdata.R)
        p_05 = (p0+pf(h,a,b,testdata.R))/2
        C0 = mattable.getC(T_new[0])
        C_05 = (C0 + mattable.getC(T_new[1])) / 2
        f0 = f(0, a, b, Tenv, testdata.R)
        f1 = f(h, a, b, Tenv, testdata.R)
        rundata.K0 = tau*(hi_05/h+h*p_05/8+h*p0/4) + h*C0/4 + h*C_05/8
        rundata.M0 = tau*(h*p_05/8 - hi_05/h) + h*C_05/8;
        rundata.P0 = testdata.F0*tau + h*tau/8*(3*f0+f1) + h*C0*TPast[0]/4 + h*C_05*(TPast[0]+TPast[1])/8

        # Right boundary conditions:
        CN = mattable.getC(T_new[n])
        CN_05 = (CN + mattable.getC(T_new[n])) / 2
        hiN_05 = hi(mattable, T_new[n-1],T_new[n])
        pN = pf(testdata.l,a,b,testdata.R)
        pN_05 = (pN+pf(testdata.l-h,a,b,testdata.R))/2
        fN0 = f(testdata.l,a,b,Tenv,testdata.R)
        fN1 = f(testdata.l-h,a,b,Tenv,testdata.R)

        rundata.KN = -tau*(hiN_05/h+alpha(testdata.l, a, b)+pN*h/4+pN_05*h/8) + h*CN/4 + h*CN_05/8;
        rundata.MN = tau*(hiN_05/h - h*pN_05/8) + h*CN_05/8;
        rundata.PN = -alpha(testdata.l,a,b)*Tenv*tau - tau*h/8*(3*fN0+fN1) + h/4*CN*TPast[n] + h/8*CN_05*(TPast[n]+TPast[n-1]);

        # Forming three-diagonal matrix method coefficients:
        x = []
        rundata.dcoef.A.append(0)
        rundata.dcoef.B.append(0)
        rundata.dcoef.C.append(0)
        rundata.dcoef.F.append(0)
        #x.append(0)
        xval = h
        for i in range(1,testdata.N):
            x.append(xval)
            rundata.dcoef.A.append(tau*hi(mattable,T_new[i-1],T_new[i])/h)
            rundata.dcoef.C.append(tau*hi(mattable,T_new[i],T_new[i+1])/h)
            rundata.dcoef.B.append(rundata.dcoef.A[-1]+rundata.dcoef.C[-1]+pf(xval,a,b,testdata.R)*h*tau+mattable.getC(T_new[i])*h)
            rundata.dcoef.F.append(mattable.getC(T_new[i])*h*TPast[i]+f(xval,a,b,Tenv,testdata.R)*h*tau)
            xval+=h
        x.append(xval)
        tdiag(rundata, testdata.N, T_new)
        dmax = 0
        imax = 0
        for i in range(0, len(T_new)):
            if (fabs(T_new[i]-T_old[i])>dmax):
                dmax = fabs(T_new[i]-T_old[i])
                imax = i
        if fabs(dmax/T_new[imax])<1e-4:
            break
    return [T_new, x]